# Lesson 3: Multi-Layered Perceptrons

## Introduction

Biological neurons are non-linear and are complex intricate networks. In machine learning, single neurons can only construct linear functions and decision boundaries. Single linear neurons can't solve XOR problems but one hidden layered MLPs can. One hidden layer MLPs can solve pratically any problem (given enough neurons in the hidden layer).

## Universal Function Approximation Theorem

The Universal Function Approximation Theorem states that feedforward neural networks with single hidden layers containing finite numbers of neurons can approximate any continuous function on a compact input domain to arbitrary accuracy given a sufficiently large number of hidden neurons and appropriate choice of activiation function.

This does beg the question of figuring out how many neurons to use. This theorem does not tell us how many hidden neurons we would need, all it's saying is that it is possible. It also doesn't tell us how to find these functions. There's no gurantee that we'll be able to find a function given a finite set of example input output pairs.

To gain some more intuition behind it, let's think about what a ReLU function is. Essentially, rectified linear units or ReLU is an activation function that introduces non-linearity to a deep learning model.

In [ ]:
import matplotlib.pyplot as plt

def relu(x):
	return max(0.0, x)

reluBase = []
for i in range(-2, 2):
    reluBase.append(relu(i))

The ReLu function takes the following shape

In [ ]:
plt.plot(reluBase)

We create some modified ReLU functions:

In [ ]:
def relu_add_1(x):
	return relu(x + 1)

def relu_x_minus_two(x):
	return -2 * max(0.0, x)

def relu_minus_1(x):
	return relu(x-1)

relu1 = []
relu2 = []
relu3 = []

Relu_Range = list(range(-3, 3))

for i in Relu_Range:
	relu1.append(relu_add_1(i))
	relu2.append(relu_x_minus_two(i))
	relu3.append(relu_minus_1(i))

fig, ax = plt.subplots()

plt.subplot(3,1,1)
plt.plot(Relu_Range, relu1)
plt.title('relu(x + 1)')

plt.subplot(3,1,2)
plt.plot(Relu_Range, relu2)
plt.title('-2 * relu(x)')

plt.subplot(3,1,3)
plt.plot(Relu_Range, relu3)
plt.title('relu(x - 1)')

plt.show()

But then when we combine these, we get:

In [ ]:
def comb_relu(x):
    return relu_add_1(x) + relu_x_minus_two(x) + relu_minus_1(x)

relu4 = []

for i in list(range(-3, 3)):
	relu4.append(comb_relu(i))

plt.plot(Relu_Range, relu4)

Here you can see that you've made a bit of a curve with a combination of ReLUs. Continuously doing this, we'll be able to replicate sin functions and create entire waves just using ReLUs.

## Building MLPs

Multilayered perceptrons are just simple linear layers stacked. The layers have activation functions or non-linearity functions labeled by $\sigma$.

For multi-class classification of `n` samples and `c` classes, we'd want to use the following loss function:
- **Cross Entropy Loss**: Compares predicted class probability to actual class desired output and then penalizes probability based on how far it is from the actual expected value. This penalty is logarithmic and yields a large score for large differences close to 1 and small score for small differences tending to 0.

\begin{equation}
\operatorname{loss}(x_i, \text { labels }_i)=-\log \left(\frac{\exp (x[\text { labels }_i])}{\sum_{j} \exp (x[j])}\right)=-x_i[\text { labels }_i]+\log \left(\sum_{j=1}^C \exp (x_i[j])\right)
\end{equation}

For a one-hidden-layer MLP with h hidden units, d inputs (features), we have a hidden layer denoted by $\bold{H} \in \mathbb{R} ^ {n \times h}$. Since the hidden and output layers are fully connected, the hidden-layer weights will be $\bold{W}^{(1)} \in \mathbb{R} ^ {d \times h}$ and biases $\bold{b}^{(1)} \in \mathbb{R} ^ {1 \times h}$. For the output-layer, we'll have weights $\bold{W}^{(2)} \in \mathbb{R} ^ {h \times q}$ and biases $\bold{b}^{(2)} \in \mathbb{R} ^ {1 \times q}$. This allows us to calculate the outputs $\bold{O} \in \mathbb{R} ^ {n \times q}$ as such:


$$
\bold{H} = \bold{XW}^{(1)} + b^{(1)}
$$

$$
\bold{O} = \bold{HW}^{(2)} + b^{(2)}
$$

With the activation functions, this becomes as follows:


$$
\bold{H} = \sigma ( \bold{XW}^{(1)} + b^{(1)})
$$

$$
\bold{O} = \bold{HW}^{(2)} + b^{(2)}
$$

Let's make a very simple MLP.

In [ ]:
import os
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 5

trainset = torchvision.datasets.CIFAR10(root='./Datasets', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./Datasets', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Flatten(),
      nn.Linear(32 * 32 * 3, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 10)
    )

  def forward(self, x):
    return self.layers(x)
  
mlp = MLP()

mlp.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)


for epoch in range(5):
  running_loss = 0.0 

  for i, data in enumerate(trainloader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    outputs = mlp(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
      print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
      running_loss = 0.0

print('Finished Training')

In [ ]:
import numpy as np
import torchvision

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print(labels)

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
outputs = mlp(images.to(device))

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))

## Activation Functions

So far we've discussed the ReLU activation function but there exist other options that we can use for activation functions. Different activation functions exist for different use cases. Each activation function also has it's own derivative which is also plotted. For the following, we make use PyTorch's built in activation functions.

### ReLU Function

In [ ]:
x = torch.arange(-8.0, 8.0, 1, requires_grad=True)
y = torch.relu(x)
plt.plot(x.detach(), y.detach())
plt.show()

Then from there, we're able to derivate the ReLU function as such:

In [ ]:
y.backward(torch.ones_like(x), retain_graph=True)
plt.plot(x.detach(), x.grad)
plt.show()

### Sigmoid Function

In [ ]:
x = torch.arange(-8.0, 8.0, 1, requires_grad=True)
y = torch.sigmoid(x)
plt.plot(x.detach(), y.detach())
plt.show()

In [ ]:
y.backward(torch.ones_like(x), retain_graph=True)
plt.plot(x.detach(), x.grad)
plt.show()

### Tanh Function

In [ ]:
x = torch.arange(-8.0, 8.0, 1, requires_grad=True)
y =  torch.tanh(x)
plt.plot(x.detach(), y.detach())
plt.show()

In [ ]:
y.backward(torch.ones_like(x), retain_graph=True)
plt.plot(x.detach(), x.grad)
plt.show()